# ----------------------------------- Projeto Cetus --------------------------------------------


### Assunto: Orçamento da Despesa Pública
### Defasagem: D-2

In [125]:
# Autor: Bruno Jardim
# Data: 10/02/2019
# Descrição: Código para salvar os dados do portal transparência no datalake (AWS - S3)
# URL: http://www.portaltransparencia.gov.br/download-de-dados

In [132]:
def CetusLakeOrcamentoDespesasGov(ANO_INFO,exec_lake = 'prod'):
    
    #---- Pagina de Extracao -----------
    # http://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/
    
    #---- Metadados do Arquivo ---------
    # http://www.portaltransparencia.gov.br/pagina-interna/603417-dicionario-de-dados-orcamento-da-despesa
    
    
    # ----- Pegando a Data da Informacao e Gravando na Tabela -------------------
    import requests
    from bs4 import BeautifulSoup
    import re
    
    if exec_lake == 'prod':
        print('CETUS --> Executando em Producao')
        # Utilizado para producao (execucao semanal ou mensal)
        URL = 'http://www.portaltransparencia.gov.br/orcamento/despesas?ordenarPor=ano&direcao=desc'
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        tableDiv = soup.find_all('div', id="datas")
        datinfo = re.sub("[^\d\.]", "", str(tableDiv))[2:]
        #print('A data da informação é: ', datinfo)
        # ----------------------------------------------------------------------------
    else:
        print('CETUS --> Executando Carga Historica')
        # Formato que pegamos no site quando no modo producao (DDMMAAAA)
        datinfo = '31' + '12' + str(ANO_INFO)
        
    
   
    import urllib.request
    urllib.request.urlretrieve('http://www.portaltransparencia.gov.br/download-de-dados/orcamento-despesa/'+str(ANO_INFO), 'dados_desp_' + str(ANO_INFO) + '.zip')

    import zipfile
    with zipfile.ZipFile('dados_desp_' + str(ANO_INFO) + '.zip',"r") as zip_ref:
        zip_ref.extractall('dados_desp_' + str(ANO_INFO))

    import pandas as pd
    import chardet
    with open('dados_desp_' + str(ANO_INFO) +'/'+str(ANO_INFO)+'_OrcamentoDespesa.zip.csv', 'rb') as f:
        result = chardet.detect(f.read())  # or readline if the file is large
    df = pd.read_csv('dados_desp_' + str(ANO_INFO) +'/'+str(ANO_INFO)+'_OrcamentoDespesa.zip.csv', encoding=result['encoding'], error_bad_lines=False,sep=';', engine='python')
    df['DT_INFO'] = datinfo
    return df


def CetusSalvaCSVS3(dataframe,dataref,nm_s3_file,s3_path):


    S3fs = s3fs.S3FileSystem()

    # Nome do arquivo a ser salvo no S3
    #nm_s3_file = 'OrcamentoDespesas_'+str(dtref)+'.csv'

    # Caminho no lake (S3)
    #s3_path = 'projeto-cetus/datalake/OrcamentoDespesas/dados/'

    bytes_to_write = dataframe.to_csv(sep = ',',header=True, index=False, index_label=None).encode()
    with S3fs.open(s3_path+nm_s3_file, 'wb') as f:
        f.write(bytes_to_write)
    return print('Salvo com Sucesso :)')



def CetusReadS3CSVFile(nm_bucket,path_file,sep=','):
    import boto3
    bucket=nm_bucket # Or whatever you called your bucket
    data_key = path_file # Where the file is within your bucket
    data_location = 's3://{}/{}'.format(bucket, data_key)
    try:
        df = pd.read_csv(data_location,sep=sep,encoding='utf-8',error_bad_lines=False) 

    except:
        df = pd.read_csv(data_location,sep=sep,encoding='latin-1',error_bad_lines=False)

    return df



### Carga histórica de 2014 a 2018

In [131]:
years = range(2014,2019)
for ano in years:
    print('Extraindo dados de: ',ano)
    dados = CetusLakeOrcamentoDespesasGov(ano,exec_lake = 'hist')
    nm_s3_file = 'OrcamentoDespesas_'+str(ano)+'.csv'
    s3_path = 'projeto-cetus/datalake/OrcamentoDespesas/dados/'
    CetusSalvaCSVS3(dados,dtref,nm_s3_file,s3_path)

Extraindo dados de:  2014
CETUS --> Executando Carga Historica


Skipping line 3762: ';' expected after '"'


Salvo com Sucesso :)
Extraindo dados de:  2015
CETUS --> Executando Carga Historica


Skipping line 4013: ';' expected after '"'


Salvo com Sucesso :)
Extraindo dados de:  2016
CETUS --> Executando Carga Historica


Skipping line 4526: ';' expected after '"'


Salvo com Sucesso :)
Extraindo dados de:  2017
CETUS --> Executando Carga Historica
Salvo com Sucesso :)
Extraindo dados de:  2018
CETUS --> Executando Carga Historica
Salvo com Sucesso :)


### Lendo dados Salvos no Lake

In [133]:
ano = 2017
nm_s3_file = 'OrcamentoDespesas_'+str(ano)+'.csv'
df = CetusReadS3CSVFile('projeto-cetus','datalake/OrcamentoDespesas/dados/'+nm_s3_file,sep=',')

In [134]:
df.head()

,EXERCÍCIO,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO SUBORDINADO,NOME ÓRGÃO SUBORDINADO,CÓDIGO UNIDADE ORÇAMENTÁRIA,NOME UNIDADE ORÇAMENTÁRIA,CÓDIGO FUNÇÃO,NOME FUNÇÃO,CÓDIGO SUBFUNÇÃO,...,CÓDIGO CATEGORIA ECONÔMICA,NOME CATEGORIA ECONÔMICA,CÓDIGO GRUPO DE DESPESA,NOME GRUPO DE DESPESA,CÓDIGO ELEMENTO DE DESPESA,NOME ELEMENTO DE DESPESA,ORÇAMENTO INICIAL (R$),ORÇAMENTO ATUALIZADO (R$),ORÇAMENTO REALIZADO (R$),DT_INFO
0,2017,20000,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,23,Comércio e serviços,691,...,4,DESPESAS DE CAPITAL,4,INVESTIMENTOS,92,DESPESAS DE EXERCICIOS ANTERIORES,"114835,52","114835,52","114835,52",31122017
1,2017,20000,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,23,Comércio e serviços,691,...,3,DESPESAS CORRENTES,3,OUTRAS DESPESAS CORRENTES,92,DESPESAS DE EXERCICIOS ANTERIORES,"460581,81","460581,81","460581,81",31122017
2,2017,20000,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,23,Comércio e serviços,691,...,3,DESPESAS CORRENTES,3,OUTRAS DESPESAS CORRENTES,39,OUTROS SERVICOS DE TERCEIROS-PESSOA JURIDICA,"5460376,54","5460376,54","5460376,54",31122017
3,2017,20000,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,23,Comércio e serviços,691,...,3,DESPESAS CORRENTES,3,OUTRAS DESPESAS CORRENTES,0,Não informado,"13103260,65","1008,65",",00",31122017
4,2017,20000,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,20101,PRESIDENCIA DA REPUBLICA,23,Comércio e serviços,691,...,4,DESPESAS DE CAPITAL,4,INVESTIMENTOS,0,Não informado,"4860945,48","100,48",",00",31122017
